In [ ]:
import os
import polars as pl
import pandas as pd
import os.path as path
from spirepy import Study
from spirepy.sample import Sample
from micom.viz import plot_growth, plot_focal_interactions, plot_mes
import gzip
from Bio import SeqIO
from glob import glob

from micom.qiime_formats import load_qiime_medium
from micom.data import test_medium
from micom.workflows import build, build_database, grow, complete_community_medium
from micom.interaction import interactions, summarize_interactions


# Sample definition and manifest generation

In [ ]:
study_name = "Lloyd-Price_2019_HMP2IBD"
study = Study(study_name)
data_folder = "/home/alexandre/Documents/projects/micro_fr_pred/data/"

In [ ]:
out_folder = path.join(data_folder,study_name,"simulations")
out_folder

In [ ]:
tax = pd.read_csv(path.join(data_folder,study_name,"study_manifest.csv"))
tax

In [ ]:
manifest = build(
    taxonomy=tax,
    out_folder=out_folder,
    model_db=None,
    cutoff=0.0001,
    threads=8
)

manifest

# Western Diet medium

In [ ]:
medium = load_qiime_medium("../data/western_diet_gut.qza")
medium

In [ ]:
res = grow(
    manifest,
    model_folder=out_folder,
    medium=medium,
    tradeoff=1.0,
    threads=8,
)

res.save(path.join(out_folder, "simulation_results.csv.zip"))

In [ ]:
pl = plot_growth(res, filename=path.join(out_folder,"growth_rates.html"))
pl.view()

In [ ]:
ints = interactions(res, taxa='spire_mag_00374978')
ints.sort_values(by="flux", ascending=False).head()

In [ ]:

full = interactions(res, taxa=None, threads=8)

full

In [ ]:

summary = summarize_interactions(full)
summary.sort_values(by="n_ints", ascending=False)

In [ ]:
summary.groupby("class")["n_ints"].sum()

In [ ]:
summary.groupby("class")["flux"].sum()

In [ ]:

pl = plot_focal_interactions(res, taxon=None, filename=path.join(out_folder, "niche.html"))
pl.view()

In [ ]:

pl = plot_mes(res, filename=path.join(out_folder, "mes.html"))
pl.view()

# Fixed medium

In [ ]:

fixed_medium = complete_community_medium(manifest, model_folder=out_folder, medium=medium,
                    community_growth=0.1, min_growth=0.01,
                    max_import=10, threads=2)
fixed_medium

In [ ]:
res_fixed = grow(
    manifest,
    model_folder=out_folder,
    medium=fixed_medium,
    tradeoff=1.0,
    threads=2,
)

In [ ]:
pl = plot_growth(res_fixed, filename=path.join(out_folder, "growth_rates_fixed_medium.html"))
pl.view()

In [ ]:

ints = interactions(res_fixed, taxa='spire_mag_00374987')
ints.sort_values(by="flux", ascending=False).head()

In [ ]:
ints = interactions(res_fixed, taxa='spire_mag_00374978')
ints.sort_values(by="flux", ascending=False).head()

In [ ]:
full_fixed = interactions(res_fixed, taxa=None, threads=8)

full_fixed

In [ ]:

summary_fixed = summarize_interactions(full_fixed)
summary_fixed.sort_values(by="n_ints", ascending=False)

In [ ]:
summary_fixed.groupby("class")["n_ints"].sum()

In [ ]:
summary_fixed.groupby("class")["flux"].sum()

In [ ]:

pl = plot_focal_interactions(res_fixed, taxon="spire_mag_00374978", filename=path.join(out_folder, "niche_fixed_medium.html"))
pl.view()

In [ ]:

pl = plot_mes(res_fixed, filename=path.join(out_folder, "mes_fixed.html"))
pl.view()